# Residential Diversity Map

## 1. Define Styling
Description

Example: https://mkz.smartuse.ch/uploads/assets/views/SU-002-Wohnungsdiversitaet.html

**Columns:**  
        *Size*: BTOT from statpop  
        *Color categories*: WT1, WT2, WT3, WT4, WT5, WT6 from gws

## 2.Cookbook

### **Spatialite Querying**  

INPUT: requested data through spatial querying  
CODE: *(see below)*  
OUTPUT: <result\>.csv  

(Assumin we have `pop` table from Residential_Density_Map)

```sqlite
SELECT load_extension('mod_spatialite');

-- QUERY BTOT FROM STATPOP
-- QUERY WT{1,2,3,4,5,6} FROM GWS
-- JOIN TABLES ON RELI
-- SPATIAL QUERY FOR SOME AREA
.header on
.mode csv
.once result.csv
SELECT A.BTOT, A.RELI, A.E_KOORD, A.N_KOORD, AsText(ST_Transform(A.geometryLV95, 4326)) AS wkt, 
B.WT1, B.WT2, B.WT3, B.WT4, B.WT5, B.WT6 from pop2 as A 
INNER JOIN gws as B 
ON A.RELI = B.RELI   
WHERE Within(A.geometryLV95,Buffer(GeomFromText('Point(2683268 1248001)'), 5000))
```

```python
>>> import pandas as pd
>>> df = pd.read_csv('result.csv')
df.dropna(inplace=True)
>>> df.to_csv('clear_result.csv', index=False)
```

### **Datapackage pipelines**

INPUT: 'clear_result.csv (from Spatialite Querying) + data_to_feed.yaml  
CODE: *(see below)*   
OUTPUT: datapackage.json, modified_result.csv  
  
CODE:

### **Exporter**

INPUT: datapackage.json, modified_result.csv  
CODE: *(see below)*   
OUTPUT: final_datapackage.json    

CODE:
```python
import yaml
import json
import geopandas as gpd

gdf = gpd.read_file('output/residential/residential.csv')
gdf.to_file("output/residential/preview.geojson", driver='GeoJSON')

with open("output/residential/datapackage.json", 'r') as j, \
     open("output/residential/preview.geojson", 'r') as l, \
     open("output/residential/final.json", 'w') as r:
    data = json.load(j)
    feed = json.load(l)
    data["resources"] = [{"name": "data-layer", "mediatype": "application/vnd.simplestyle-extended", "data": {"name": "data", "type": "FeatureCollection", "features": []}},{"name": "mapbox-background", "path": "mapbox://styles/gemeindescan/ck6rp249516tg1iqkmt48o4pz", "mediatype": "application/vnd.mapbox-vector-tile", "data":"" }]
    data['resources'][0]['data']['features'] = feed['features']
    json.dump(data, r)
```